# Advection-Diffusion 

In this tutorial we present how to solve the unsteady advection-diffusion problem [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unknown is the mass $u$.

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$v$ the velocity field, $D$ being the constant diffusion coefficient and $S$ being the source term, we want to solve the following problem: find $u$ such that
$$
\partial_t u - \nabla \cdot (D \nabla u - a \cdot u) = S
$$

$$
\partial_t u - D \nabla^2 u + a \cdot \nabla u = S
$$
Multiply by test (weight) function and integrate over domain 
$$
\int_\Omega \partial_t u \varphi d\Omega - \int_\Omega D \nabla^2 u \varphi d\Omega + \int_\Omega a \cdot \nabla u \varphi d\Omega = \int_\Omega S \varphi d\Omega
$$
We use integration by parts to reduce the order by moving gradient to the test function.
$$
\int_\Omega \partial_t u \varphi d\Omega + \int_\Omega D \nabla u \cdot \nabla \varphi d\Omega - \int_\Gamma D \nabla u \cdot \nu \varphi d\Gamma + \int_\Omega a \cdot \nabla u \varphi d\Omega = \int_\Omega S \varphi d\Omega
$$
Discretize in time with backwards Euler
$$
\int_\Omega \frac{u^{n+1} - u^n}{\tau} \varphi d\Omega + D\int_\Omega \nabla u^{n+1} \cdot \nabla \varphi d\Omega - D\int_\Gamma \nabla u^{n+1} \cdot \nu \varphi d\Gamma + \int_\Omega a \cdot \nabla u^{n+1} \varphi d\Omega = \int_\Omega S^{n+1} \varphi d\Omega
$$
Move the known, boundary and source term to the rhs
$$
\int_\Omega u^{n+1} \varphi d\Omega + \tau D\int_\Omega \nabla u^{n+1} \cdot \nabla \varphi d\Omega + \tau \int_\Omega (a \cdot \nabla u^{n+1}) \varphi d\Omega = \tau \int_\Omega S^{n+1} \varphi d\Omega + \int_\Omega u^n \varphi d\Omega + \tau D\int_\Gamma (\nabla u^{n+1} \cdot \nu) \varphi d\Gamma
$$
Gives us in matrix form
$$
[M^{n+1} + \tau (D + A)](u^{n+1}) = \tau S^{n+1} + M^{n}u^n + \tau BC^{n+1}
$$


with boundary conditions:
$$ u = 0 \text{ on } \partial_{top} \Omega \qquad u = 1 \text{ on } \partial_{bottom} \Omega \qquad \nu \cdot u = 0 \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega$$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
import os
import shutil
import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

import inspect

We create now the grid, since we will use a Lagrangian of order 1 to approximation for ${u}$ we are restricted to simplices. In this example we consider a 2-dimensional structured grid, but the presented code will work also in 1d and 3d. PyGeoN works with mixed-dimensional grids, so we need to convert the grid.

In [2]:
grid_size = [5, 5]      # n x n grid 
dim = [1,1]    
dt = 0.1    # time step size
num_steps = 10      # number of time steps
key = "mass"    # key for the unknown variable

# Create a structured triangle grid with given size and dimension
sd = pp.StructuredTriangleGrid(grid_size, dim)
# Convert the grid into a mixed-dimensional grid
mdg = pg.as_mdg(sd)

# Convert to the PorePy grid object to a PyGeoN grid object
pg.convert_from_pp(sd)
sd.compute_geometry()

In [3]:
def first_order_tensor(grid,
        vx: np.ndarray,
        vy: np.ndarray = None,
        vz: np.ndarray = None,
    ):
    
        n_cells = vx.size
        vel = np.zeros((3, n_cells))

        vel[0, ::] = vx
             
        if vy is not None:
            vel[1, ::] = vy

        if vz is not None:
            vel[2, ::] = vz

        return vel  

With the following code we set the data, in particular the velocity field, diffusion coefficient and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

We define the source term function. Given below are two source functions: a point source, and a gaussian source at the center cell of the grid.

In [4]:
def source_term_gauss(x):
    # Example: a Gaussian source at the center of the grid
    center = sd.cell_centers[:,sd.num_cells // 2]
    sigma = 0.05
    r2 = np.sum((x - center)**2)
    return np.exp(-r2 / (2 * sigma**2))

def source_term_point(x):
    # Example: a point source at the center of the grid

    # Define the center cell
    center_cell = sd.num_cells // 2 - 1 
    # Get the boundary nodes of the grid
    bd_nodes = sd.get_all_boundary_nodes()
    # A map from cell to nodes
    node_map = sd.cell_nodes()
    # Get the nodes of the center cell
    center_nodes = node_map[:,center_cell].nonzero()[0]
    # Filter out the boundary nodes from the center nodes
    mask = ~np.isin(center_nodes, bd_nodes)
    center_nodes = center_nodes[mask]
    # Get the coordinates of the center nodes
    source_nodes_coord = sd.nodes.T[center_nodes]

    # Check if the input x matches any of the center nodes' coordinates
    return 1.0 if np.any(np.all(source_nodes_coord == x, axis=1)) else 0.0

Define the natural boundary condition function

In [5]:
def u_bc(x):
    # Example: Neumann boundary condition with inflow at the left and zero 
    # outflow elsewhere
    return 1.0 if abs(x[0]) < 1e-10 else 0.0

In [ ]:
key = "mass"

nat_bc_val = []
ess_bc = []
source = []
sol = []

# Use first order Lagrange elements to approximate the solution
P1 = pg.Lagrange1(key)

# Loop through each subdomain in the mixed-dimensional grid
# (NB: only one subdomain in this case)
for sd, data in mdg.subdomains(return_data=True):
    # Set the values for the diffusion tensor and velocity field parameter 
    diff = pp.SecondOrderTensor(np.ones(sd.num_cells))
    vel_field = first_order_tensor(sd, np.ones(sd.num_cells))
    # Store the parameters in the subdomain data
    param = {"first_order_tensor": vel_field, "second_order_tensor": diff}
    pp.initialize_data(sd, data, key, param)

    # Identify boundary faces
    left = sd.face_centers[0, :] == 0
    right = sd.face_centers[0, :] == 1
    bottom = sd.nodes[1, :] == 0
    top = sd.nodes[1, :] == 1
    
    # Define natural and essential boundary faces 
    nat_bc_faces = np.logical_or(left, right)
    ess_bc_faces = np.logical_or(bottom, top)
    
    # Set values for the natural boundary condition
    nat_bc_val.append(dt * P1.assemble_nat_bc(sd, u_bc, nat_bc_faces))

    # Set boolean array for nodes which are essential boundary condition
    ess_bc.append(ess_bc_faces)

    # Set the source term values 
    mass = P1.assemble_mass_matrix(sd)
    source.append(dt * mass @ P1.interpolate(sd, source_term_point))

Once the data are assigned to the mixed-dimensional grid, we construct the matrices. In particular, the linear system given in above. Once the matrix is created, we also construct the right-hand side containing the boundary conditions.

In [ ]:
# Construct the local matrices
mass = P1.assemble_mass_matrix(sd)
adv = P1.assemble_adv_matrix(sd, data)
stiff = P1.assemble_stiff_matrix(sd, data)

# Assemble the global matrix
# fmt: off
global_matrix = mass + dt*(adv + stiff)
# fmt: on

# Get the degrees of freedom for u
dof_u = P1.ndof(sd)

# Assemble the time-independent right-hand side
rhs_const = np.zeros(dof_u)
rhs_const[:dof_u] += np.hstack(nat_bc_val) + np.hstack(source)

# Set initial conditions
u = np.zeros(dof_u)

# Add the initial condition to the time series
sol.append(u)

We need to solve the linear system, PyGeoN provides a framework for that. The actual imposition of essential boundary conditions (flux boundary conditions) might change the symmetry of the global system, the class `pg.LinearSystem` preserves this structure by internally eliminating these degrees of freedom. Once the problem is solved, we extract the solutions $u$

In [ ]:
for n in np.arange(num_steps):

    # Update the right-hand side with the current solution
    rhs = rhs_const.copy()
    rhs[:dof_u] += np.hstack(mass @ u)

    # Set up the linear solver
    ls = pg.LinearSystem(global_matrix, rhs)

    # Flag the essential boundary conditions
    ls.flag_ess_bc(np.hstack(ess_bc), np.zeros(dof_u))

    # Solve the linear system
    x = ls.solve()

    # Extract the variables
    u = x[:dof_u]
    
    # Add the solution to the time series
    sol.append(u)

Since the computed $u$ is one value per node of the grid, for visualization purposes we project the mass in each cell center. We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [ ]:
output_directory = os.path.join(os.path.dirname(__file__), "adv-diff sol")
# Delete the output directory, if it exisis
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

save = pp.Exporter(mdg, "adv-diff", folder_name=output_directory)

proj_u = P1.eval_at_cell_centers(sd)

for n, u in enumerate(sol):
    for _, data in mdg.subdomains(return_data=True):
        
        # post process variables
        cell_u = proj_u @ u

        pp.set_solution_values("mass", cell_u, data, time_step_index=0)
        save.write_vtu(["mass"], time_step=n)

save.write_pvd(range(len(sol)))